In [1]:
# bfs concurrency
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
import threading

def bfs(url):
    visited = set([url])
    dq = deque([[url, "", 0]])
    max_depth = 3
    max_breadth = 5
    string = ""
    while dq:
        base, path, depth = dq.popleft()
        if depth < max_depth:
            try:
                soup = BeautifulSoup(requests.get(base + path).text, "html.parser")
                links = soup.find_all("a")[:max_breadth]
                for link in links:
                    href = link.get("href")
                    if href not in visited:
                        visited.add(href)
                        string += ("  " * depth + f"at depth {depth}: {href}\n")

                        if href.startswith("http") or href.startswith("https"):
                            dq.append([href, "", depth + 1])
                        else:
                            dq.append([base, href, depth + 1])
            except:
                pass

    filename =""
    if "https://" in url:
        filename = re.sub(r"https://","",url)
        filename = re.sub(r".com","",filename)
    elif "http://" in url:
        filename = re.sub(r"http://","",url)
        filename = re.sub(r".com","",filename)

    file  = open("crawled_links/"+filename+".txt","w")
    file.write(string)
    file.close()

In [2]:
frontier = ["http://toscrape.com", "https://soundcloud.com", "http://reddit.com", "https://fc2.com"]

def threadCreater(frontier):
    threads = []
    for i in frontier:
        t = threading.Thread(target = bfs, args=(i,))
        threads.append(t)
    return threads

threads = threadCreater(frontier)
for i in threads:
    i.start()
for i in threads:
    i.join()
print("files made!")

files made!


In [3]:
import math

def unary(x):
    return '0'*(x-1)+'1'

def encoder(x, b):
    q = math.floor(x/b)
    q_encode = unary(q+1)
    i = math.floor(math.log2(b))
    d = int(2**(i+1))-b
    r = x%b
    rem = ""
    if r < d:
        rem = bin(r)[2:]
        l = len(rem)
        if l < i:
            rem = '0'*(i - l)+rem
    else:
        rem = bin(r + d)[2:]
        l = len(rem)
        if l < i+1:
            rem = '0'*(i+1-l) + rem
    return q_encode+rem

print(encoder(74, 16))
print(encoder(50, 11))

000011010
000011011


In [4]:
def decode(n, M):
    q=len(n.split('1')[0])
    b=math.floor(math.log2(M))
    k = 2**(b+1)-M
    r = int(n[q+1:q+1+b],2)
    if r>=k:
        r = int(n[q+1:q+1+b+1],2)
        r=r-k
    x=q*M+r
    return x

print(decode("1111111110010001101", 10))

9


In [5]:
# Q4
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from functools import reduce
from urllib import request
import pandas as pd

stop_words = set(STOP_WORDS)
stop_words = stop_words.union({'.',',','\'','\"','?','{','}','[',']','<','>','(',')','!'})

def extract(url):
    html = request.urlopen(url).read().decode('utf8')
    string = BeautifulSoup(html, 'html.parser').get_text().lower()
    return list(word_tokenize(string))

def store(name, ls):
    with open(name,'w') as f:
        for item in list(ls):
            try:
                f.write('%s\n'%item)
            except:
                pass
def term_maker(ls, filenames):
    terms = []
    index = 0
    for i in ls:
        st = set(i).difference(stop_words)
        store('extracted_content/'+filenames[index]+'.txt', list(st))
        index += 1
        terms.append(list(st))
    terms = reduce(lambda z,y : z+y, terms)
    return terms

def counter(tokens, terms):
    ls = []
    for i in terms:
        ls.append(tokens.count(i))
    return ls

base = 'https://en.wikipedia.org/wiki/'
links = ['Web_mining','Data_mining','Artificial_intelligence','Machine_learning','Natural_language_processing','Text_mining']
token_list = []
for i in links:
    token_list.append(extract(base+i))
terms = term_maker(token_list, links)
bagOfWords = []
for i in token_list:
    bagOfWords.append(counter(i,terms))

pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : bagOfWords[0],     
    "Doc 2" : bagOfWords[1],     
    "Doc 3" : bagOfWords[2],     
    "Doc 4" : bagOfWords[3],     
    "Doc 5" : bagOfWords[4],     
    "Doc 6" : bagOfWords[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,algorithms,5,13,23,56,12,3
1,demonstrated,1,0,3,0,0,2
2,statistics,2,9,6,19,3,1
3,trend,2,0,0,0,0,0
4,points,1,0,1,1,0,2
...,...,...,...,...,...,...,...
15125,sanger,0,1,0,0,0,2
15126,university,0,4,22,6,5,9
15127,email,0,0,1,3,0,2
15128,traditional,2,0,4,1,0,1


In [6]:
def total(ls):
    count=0
    for i in ls:
        if i != 0:
            count+=1
    return count

TF = [[number/total(i) for number in i] for i in bagOfWords]
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF[0],     
    "Doc 2" : TF[1],     
    "Doc 3" : TF[2],     
    "Doc 4" : TF[3],     
    "Doc 5" : TF[4],     
    "Doc 6" : TF[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,algorithms,0.001440,0.002295,0.002095,0.007223,0.002234,0.000581
1,demonstrated,0.000288,0.000000,0.000273,0.000000,0.000000,0.000387
2,statistics,0.000576,0.001589,0.000547,0.002451,0.000559,0.000194
3,trend,0.000576,0.000000,0.000000,0.000000,0.000000,0.000000
4,points,0.000288,0.000000,0.000091,0.000129,0.000000,0.000387
...,...,...,...,...,...,...,...
15125,sanger,0.000000,0.000177,0.000000,0.000000,0.000000,0.000387
15126,university,0.000000,0.000706,0.002004,0.000774,0.000931,0.001744
15127,email,0.000000,0.000000,0.000091,0.000387,0.000000,0.000387
15128,traditional,0.000576,0.000000,0.000364,0.000129,0.000000,0.000194


In [7]:
import math
def IDF_helper(ls):
    total = []
    length = len(ls[0])
    for i in range(length):
        count = 0
        for j in range(len(ls)):
            value = ls[j][i]
            if value != 0:
                count += 1
        total.append(count)
    return total

def IDF_calc():
    helper = IDF_helper(bagOfWords)
    no_of_doc = 6
    IDF_ls = []
    for i in helper:
        IDF_ls.append(math.log((1+no_of_doc)/i))
    return IDF_ls
pd.DataFrame({
    "Terms" : list(terms),
    "IDF value" : IDF_calc()
})

,Terms,IDF value
0,algorithms,0.154151
1,demonstrated,0.847298
2,statistics,0.154151
3,trend,1.945910
4,points,0.559616
...,...,...
15125,sanger,1.252763
15126,university,0.336472
15127,email,0.847298
15128,traditional,0.559616


In [8]:
def TF_IDF_calc(ls):
    IDF = IDF_calc()
    for i in range(len(ls[0])):
        for j in range(6):
                ls[j][i]*=IDF[i]
    return ls
TF_IDF = TF_IDF_calc(TF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF_IDF[0],
    "Doc 2" : TF_IDF[1],
    "Doc 3" : TF_IDF[2],
    "Doc 4" : TF_IDF[3],
    "Doc 5" : TF_IDF[4],
    "Doc 6" : TF_IDF[5]
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,algorithms,0.000222,0.000354,0.000323,0.001113,0.000344,0.000090
1,demonstrated,0.000244,0.000000,0.000232,0.000000,0.000000,0.000328
2,statistics,0.000089,0.000245,0.000084,0.000378,0.000086,0.000030
3,trend,0.001121,0.000000,0.000000,0.000000,0.000000,0.000000
4,points,0.000161,0.000000,0.000051,0.000072,0.000000,0.000217
...,...,...,...,...,...,...,...
15125,sanger,0.000000,0.000221,0.000000,0.000000,0.000000,0.000485
15126,university,0.000000,0.000238,0.000674,0.000260,0.000313,0.000587
15127,email,0.000000,0.000000,0.000077,0.000328,0.000000,0.000328
15128,traditional,0.000322,0.000000,0.000204,0.000072,0.000000,0.000108


In [9]:
def query_TF_IDF(string):
    ref_tokens = list(word_tokenize(string.lower()))
    tokens = list(set(word_tokenize(string.lower())).difference(stop_words))
    Tf_query = []
    for i in tokens:
        Tf_query.append(1/ref_tokens.count(i))
    ref_IDF = IDF_calc()
    TF_IDF = []
    for i in range(len(tokens)):
        if tokens[i] in terms:
            index = terms.index(tokens[i])
            value = ref_IDF[index]*Tf_query[i]
            TF_IDF.append(value)
        else:
            TF_IDF.append(0)
    return (tokens,TF_IDF)

string = "I want communications based on machine learning"
Q_TF_IDF = query_TF_IDF(string)
pd.DataFrame({
    "Terms" : Q_TF_IDF[0],
    "TF_IDF" : Q_TF_IDF[1]
})

,Terms,TF_IDF
0,communications,1.252763
1,learning,0.154151
2,want,1.945910
3,machine,0.154151
4,based,0.154151


In [10]:
def normalization_Document(TF_IDF):
    no_of_docs = len(TF_IDF)
    for i in range(no_of_docs):
        divider = 0
        for j in TF_IDF[i]:
            divider += j**2
        divider = math.sqrt(divider)
        TF_IDF[i] = [k/divider for k in TF_IDF[i]]
    return TF_IDF

normalized = normalization_Document(TF_IDF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : normalized[0],
    "Doc 2" : normalized[1],
    "Doc 3" : normalized[2],
    "Doc 4" : normalized[3],
    "Doc 5" : normalized[4],
    "Doc 6" : normalized[5],
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,algorithms,0.007011,0.011187,0.004262,0.031093,0.010393,0.003176
1,demonstrated,0.007707,0.000000,0.003056,0.000000,0.000000,0.011639
2,statistics,0.002804,0.007745,0.001112,0.010549,0.002598,0.001059
3,trend,0.035399,0.000000,0.000000,0.000000,0.000000,0.000000
4,points,0.005090,0.000000,0.000673,0.002016,0.000000,0.007687
...,...,...,...,...,...,...,...
15125,sanger,0.000000,0.006994,0.000000,0.000000,0.000000,0.017209
15126,university,0.000000,0.007513,0.008898,0.007272,0.009452,0.020799
15127,email,0.000000,0.000000,0.001019,0.009156,0.000000,0.011639
15128,traditional,0.010180,0.000000,0.002691,0.002016,0.000000,0.003844


In [11]:
def normalization_Query(string):
    TF_IDF = query_TF_IDF(string)[1]
    divider = 0
    for i in TF_IDF:
        divider += i**2
    TF_IDF = [i/divider for i in TF_IDF]
    return TF_IDF

pd.DataFrame({
    "Terms" : query_TF_IDF(string)[0],
    "TF_IDF" : normalization_Query(string)
})

,Terms,TF_IDF
0,communications,0.230828
1,learning,0.028403
2,want,0.358543
3,machine,0.028403
4,based,0.028403


In [12]:
def cosine_dist():
    TF_IDF_doc = normalization_Document(TF_IDF)
    TF_IDF_query = normalization_Query(string)
    tokens = Q_TF_IDF[0]
    index_ls = []
    for i in tokens:
        index_ls.append(terms.index(i))
    
    cosine_dict = {}
    for i in range(len(TF_IDF_doc)):
        cosine_dist = 0
        index = 0
        for j in index_ls:
            cosine_dist += TF_IDF_doc[i][j]* TF_IDF_query[index]
            index+=1
        cosine_dict['Doc'+str(i+1)] = cosine_dist
    return cosine_dict

pd.DataFrame(list(cosine_dist().items()), columns=['Documents', 'Cosine Dist'])

,Documents,Cosine Dist
0,Doc1,0.005778
1,Doc2,0.002249
2,Doc3,0.003736
3,Doc4,0.007917
4,Doc5,0.001648
5,Doc6,0.000662


In [13]:
def euclidean_dist():
    TF_IDF_doc = normalization_Document(TF_IDF)
    TF_IDF_query = normalization_Query(string)
    tokens = Q_TF_IDF[0]
    index_ls = []
    for i in tokens:
        index_ls.append(terms.index(i))
    
    euclidean_dict = {}
    for i in range(len(TF_IDF_doc)):
        euclidean_dist = 0
        index = 0
        for j in index_ls:
            euclidean_dist += (TF_IDF_doc[i][j] - TF_IDF_query[index])**2
            index+=1
        euclidean_dist = math.sqrt(euclidean_dist)
        euclidean_dict['Doc'+str(i+1)] = euclidean_dist
    return euclidean_dict

pd.DataFrame(list(euclidean_dist().items()), columns=['Documents', 'Euclidean Dist'])

,Documents,Euclidean Dist
0,Doc1,0.416352
1,Doc2,0.427644
2,Doc3,0.421194
3,Doc4,0.456539
4,Doc5,0.426861
5,Doc6,0.427928


In [14]:
sorted_dict = {k: v for k,v in sorted(cosine_dist().items(), key=lambda item: item[1])}

pd.DataFrame(list(sorted_dict.items()))

,0,1
0,Doc6,0.000662
1,Doc5,0.001648
2,Doc2,0.002249
3,Doc3,0.003736
4,Doc1,0.005778
5,Doc4,0.007917


In [15]:
def doc_similarity():
    TF_IDF_doc = normalization_Document(TF_IDF)
    len_doc = len(TF_IDF_doc[0])
    cosine_sim = []
    for i in range(1, len(TF_IDF_doc)):
        temp = 0
        for j in range(len_doc):
            temp += TF_IDF_doc[0][j] * TF_IDF_doc[i][j]
        cosine_sim.append(temp)
    return cosine_sim

ls = ["Doc1 and Doc2", "Doc1 and Doc3", "Doc1 and Doc4", "Doc1 and Doc5", "Doc1 and Doc6"]

pd.DataFrame({
    "Similarity between" : ls,
    "Cosine value" : doc_similarity()
})

,Similarity between,Cosine value
0,Doc1 and Doc2,0.369324
1,Doc1 and Doc3,0.135916
2,Doc1 and Doc4,0.182782
3,Doc1 and Doc5,0.145552
4,Doc1 and Doc6,0.277352
